# Extraindo empresas

In [10]:
import requests
import yfinance as yf
import pandas as pd
import os

# --- FUNÇÃO PRINCIPAL SEM FILTROS ---
def scanner_completo():
    """
    Busca TODOS os ativos da API brapi e coleta suas informações detalhadas
    no yfinance, sem aplicar filtros de setor, país ou market cap.
    """
    print("Iniciando scanner COMPLETO da B3 (sem filtros)...")
    print("AVISO: Este processo pode ser demorado.")
    try:
        url = "https://brapi.dev/api/quote/list"
        lista_completa_brapi = requests.get(url).json().get('stocks', [])
    except Exception as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    # Etapa 1: Mapear TODOS os tickers para seus setores
    print("Mapeando todos os ativos encontrados...")
    tickers_para_analisar = {}
    for stock in lista_completa_brapi:
        ticker = f"{stock.get('stock')}.SA"
        # Usamos 'N/A' se o setor não for informado na brapi
        setor_brapi = stock.get('sector', 'N/A')
        tickers_para_analisar[ticker] = setor_brapi
    
    total = len(tickers_para_analisar)
    print(f"✅ Mapeamento concluído. {total} ativos serão analisados em detalhe.")

    # Etapa 2: Coleta de Dados Detalhada
    dados_finais = {}
    print("\nIniciando coleta detalhada de dados no yfinance...")
    for i, (ticker, setor_brapi) in enumerate(tickers_para_analisar.items()):
        try:
            # Imprime o progresso a cada 10 tickers para não poluir a tela
            if (i + 1) % 10 == 0:
                print(f"Analisando [{i+1}/{total}]: {ticker}...")

            stock = yf.Ticker(ticker)
            info = stock.info
            
            # Todos os filtros foram removidos conforme solicitado.
            # Apenas o try/except permanece para lidar com erros de tickers inválidos.

            dados_finais[ticker] = {
                "Empresa": info.get("longName", "N/A"),
                "Setor (brapi)": setor_brapi,
                "Subsetor (yfinance)": info.get("industry", "N/A"),
                "País": info.get("country", "N/A"),
                "Market Cap": info.get("marketCap"),
                # <<< NOVA COLUNA ADICIONADA AQUI >>>
                "Logo": info.get("logo_url", "N/A"),
            }
        except Exception:
            # Ignora silenciosamente qualquer erro para um ticker específico e continua
            continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_completo()

if resultados:
    print("\n✅ Análise completa finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    df_resultado.reset_index(inplace=True)
    df_resultado.rename(columns={'index': 'Ticker'}, inplace=True)
    
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(
             lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if pd.notna(x) and x > 0 else "N/A"
         )

    # Ordena pelo Market Cap do maior para o menor como critério principal
    df_ordenado = df_resultado.sort_values(by=['Market Cap Num'], ascending=False)
    
    # Define a ordem final das colunas para o relatório
    colunas_finais = ['Ticker', 'Empresa', 'Setor (brapi)', 'Subsetor (yfinance)', 'País', 'Market Cap', 'Logo']
    df_final = df_ordenado[colunas_finais]
    
    print("\n--- Relatório Completo da B3 ---")
    print(df_final.to_string(index=False))

    # --- SALVANDO O ARQUIVO NO CAMINHO ESPECÍFICO ---
    try:
        output_folder = r"E:\Github\finance-manager\datasets"
        output_filename = "scanner_completo.xlsx" # Novo nome do arquivo
        full_path = os.path.join(output_folder, output_filename)
        
        os.makedirs(output_folder, exist_ok=True)
        
        df_final.to_excel(full_path, index=False)
        print(f"\n✅ Resultados salvos com sucesso no caminho: {full_path}")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo Excel: {e}")
else:
    print("\nNenhuma empresa foi encontrada ou processada com sucesso.")

Iniciando scanner COMPLETO da B3 (sem filtros)...
AVISO: Este processo pode ser demorado.
Mapeando todos os ativos encontrados...
✅ Mapeamento concluído. 1705 ativos serão analisados em detalhe.

Iniciando coleta detalhada de dados no yfinance...
Analisando [10/1705]: PETR4.SA...
Analisando [20/1705]: LREN3.SA...
Analisando [30/1705]: PRIO3.SA...
Analisando [40/1705]: BBDC3.SA...
Analisando [50/1705]: EQTL3.SA...
Analisando [60/1705]: HAPV3.SA...
Analisando [70/1705]: ELET3.SA...
Analisando [80/1705]: VIVA3.SA...
Analisando [90/1705]: YDUQ3.SA...
Analisando [100/1705]: FLRY3.SA...
Analisando [110/1705]: SAPR11.SA...
Analisando [120/1705]: LIGT3.SA...
Analisando [130/1705]: AZEV3.SA...
Analisando [140/1705]: EZTC3.SA...
Analisando [150/1705]: GRND3.SA...
Analisando [160/1705]: ELET6.SA...
Analisando [170/1705]: SHUL4.SA...
Analisando [180/1705]: CSED3.SA...
Analisando [190/1705]: AURA33.SA...
Analisando [200/1705]: NASD11.SA...
Analisando [210/1705]: MELI34.SA...
Analisando [220/1705]: 

In [24]:

df_dados = pd.read_excel(r'E:\Github\finance-manager\datasets\scanner_completo.xlsx')


df_dados = df_dados[df_dados['Ticker'] == 'TAEE4.SA']

# Filter rows where 'Setor (brapi)' is one of the specified values
df_dados = df_dados[df_dados['Setor (brapi)'].isin(["Finance", "Utilities", "Communications", "Industrial Services"])]

# Filter rows where País equals 'Brazil'
# df_dados = df_dados[df_dados['País'] == 'Brazil']


print(df_dados)


       Ticker                                        Empresa  \
408  TAEE4.SA  Transmissora Aliança de Energia Elétrica S.A.   

           Setor (brapi)             Subsetor (yfinance)    País   Market Cap  \
408  Industrial Services  Utilities - Regulated Electric  Brazil  R$ 11.87 Bi   

     Logo  
408   NaN  
